In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import sys, os, re, csv, codecs
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import math
import random
from collections import defaultdict
from pprint import pprint
# Prevent future/deprecation warnings from showing in output
import warnings
warnings.filterwarnings(action='ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from itertools import product
import numpy

In [ ]:
data =pd.read_csv('file path', encoding='utf-8', usecols=['col1', 'col2'])

In [ ]:
data.head()

In [ ]:
data.sentiment.value_counts()

In [ ]:
X = data.full_text
y = data.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

In [ ]:
len(X_test)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
# Removing stop words
def get_stop_words(path):
    #"stop_words.txt"
    stop_words = []
    with codecs.open(path, "r", encoding="utf-8", errors="ignore") as myfile:
        stop_words = myfile.readlines()
    stop_words = [word.strip() for word in stop_words]
    return stop_words
stop_words = get_stop_words('/content/stop_words_arabic.txt')

from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])

In [ ]:
vect = CountVectorizer(max_features=1000, binary=True)
X_train_vect = vect.fit_transform(X_train)

In [ ]:
sm = SMOTE()
X_train_res, y_train_res = sm.fit_sample(X_train_vect, y_train)

In [ ]:
unique, counts = np.unique(y_train_res, return_counts=True)
print(list(zip(unique, counts)))

In [ ]:
#This is the fit score, and not the actual accuracy score
nb = MultinomialNB()
nb.fit(X_train_res, y_train_res)
nb.score(X_train_res, y_train_res)

In [ ]:
X_test_vect = vect.transform(X_test)
y_pred = nb.predict(X_test_vect)
y_pred

In [ ]:
clf= MultinomialNB()
clf.fit(X_train_res, y_train_res)

score = clf.score(X_train_res, y_train_res)
print("score of Naive Bayes algo is :" , score)

y_pred = clf.predict(X_test_vect)
print(confusion_matrix(y_test,y_pred))

print("Precision Score : ",precision_score(y_test, y_pred, 
                                           average='micro'))
print("Recall Score : ",recall_score(y_test, y_pred, 
                                           average='micro'))

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, numpy.newaxis]
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = numpy.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.show()

In [ ]:
class_labels = ['Positive', 'Negative', 'Neutral']
cm = confusion_matrix(y_test,y_pred).T
plot_confusion_matrix(cm, classes = class_labels)

**Evaluation Metrics for Classification**

In [ ]:
mat = confusion_matrix(y_test,y_pred)
# Confusion Matrix
print(mat)
# Classification Report
print(classification_report(y_test,y_pred))
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, xticklabels = True, yticklabels = True)